In [41]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

columns = ['data_inversa', 'mortos', 'municipio', 'uf', 'pessoas' ]
df = pd.DataFrame([], columns = columns)
for i,ano in enumerate(np.arange(2007,2022)):
    data = pd.read_csv("../Detram/datatran" + str(ano) + '.csv', delimiter = ';',  low_memory=False)
    df1 = data[columns]
    df = pd.concat([df,df1])



In [42]:
df = pd.DataFrame(df)
df['data_inversa'] = pd.to_datetime(df['data_inversa'], format =  '%d/%m/%Y' )
df['pessoas'] = pd.to_numeric(df['pessoas'])
df['mortos'] = pd.to_numeric(df['mortos'])

df['acidentes'] = 1

n = df[df['uf'] =='(null)' ].index
df = df.drop(n)
df = df.rename({'data_inversa': 'Data', 'municipio': 'Municipio', 'pessoas': 'Pessoas', 'mortos': 'Mortos', 'acidentes': 'Acidente', 'uf': 'Estado'}, axis='columns')

#df.to_excel(r'./dataset.xlsx')
#data = pd.read_csv('./dataset.csv')

In [43]:
df['Ano'] = df['Data'].dt.year  

In [44]:
df['Mês'] = df['Data'].dt.strftime('%B')

In [45]:
df = df.sort_values(by=['Estado', 'Municipio', 'Ano', 'Mês']) 

In [46]:
df1 = df.groupby(by=['Estado','Ano', 'Mês']).agg('sum')

In [51]:
df1 = df.groupby(by=['Estado','Ano', 'Mês']).agg('sum').reset_index()

# Acidentes Rodoviários 

 
Conforme dados coletados no site do Detram, podemos visualizar várias informações sobre a ocorrência dos acidentes nas rodovias brasileiras, os dados coletados e informados a população são de grande importância para criação de alternativas preventivas para a diminuição de acidentes e consequentemente na diminuição de óbitos nas estradas. 
Nos gráficos a seguir podemos visualizar duas situações, primeira mostra a quantidade de acidentes por estado e a segunda podemos verificar a quantidade de acidentes por município de cada estado. 


In [48]:
Estados = df1.Estado.unique()
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="selectstate",
        options=[{"label": x, "value": x} for x in Estados]
    ),
    dcc.Graph(id="bar-chart"),
])

                  
@app.callback(
    Output("bar-chart", "figure"), 
    [Input("selectstate", "value")])
def update_bar_chart(estado):
    fig = px.bar(df1[df1["Estado"] == estado], x="Ano", y="Acidente", title="Quantidade de Acidentes por Estado", color = 'Mês')
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline', port=8490)



In [49]:
df2 = df.groupby(by=['Estado','Ano','Municipio','Mês']).agg('sum').reset_index()

In [77]:
Estados = df2.Estado.unique()
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="selectstate",
        options=[{"label": x, "value": x} for x in Estados]
    ),
    html.Hr(),
    dcc.Dropdown(id="selectcountry"),
    html.Hr(),
    dcc.Graph(id="bar-chart"),
])

                  

@app.callback(
    Output('selectcountry', 'options'),
    Input('selectstate', 'value'))
def set_cities_options(selected_stage):
    return [{'label': i, 'value': i} for i in df2[df2['Estado']==selected_stage].Municipio.unique()]


@app.callback(
    Output("bar-chart", "figure"), 
    [Input("selectcountry", "value")])
def update_bar_chart(municipio):
    fig = px.bar(df2[df2["Municipio"] == municipio], x="Ano", y="Acidente", title="Quantidade de Acidentes por Municipio", color='Mês')
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, mode='inline', port=8590)



C:\Users\leona\anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [90]:
df3 = df.groupby(by=['Ano','Estado']).agg('sum').reset_index()

In [102]:
bubble = px.scatter(df3, x="Acidente", y="Mortos", animation_frame="Ano", animation_group="Ano", color = "Estado",
           size="Mortos",range_x=[0,max(df3['Acidente'].values)+20], range_y=[0,max(df3['Mortos'].values)+20],
           title="Diagrama de Dis",
                 )
bubble.update_yaxes(nticks=5)
bubble.update_xaxes(nticks=5)
bubble.show()

In [65]:
df['Data'].dt.strftime('%d, %b , %Y')

88774    08, Apr , 2007
78809    26, Aug , 2007
7875     26, Jan , 2007
11829    31, Jan , 2007
66245    20, Jul , 2007
              ...      
30304    24, Jun , 2019
3079     18, Jan , 2020
24950    08, Jun , 2020
46083    05, Oct , 2020
2453     15, Jan , 2021
Name: Data, Length: 1877446, dtype: object

In [63]:
df

,Data,Mortos,Municipio,Estado,Pessoas,Acidente,Ano,Mês
88774,2007-04-08,1,ACRELANDIA,AC,4,1,2007,April
78809,2007-08-26,0,ACRELANDIA,AC,1,1,2007,August
7875,2007-01-26,0,ACRELANDIA,AC,1,1,2007,January
11829,2007-01-31,0,ACRELANDIA,AC,1,1,2007,January
66245,2007-07-20,0,ACRELANDIA,AC,2,1,2007,July
...,...,...,...,...,...,...,...,...
30304,2019-06-24,1,XAMBIOA,TO,2,1,2019,June
3079,2020-01-18,0,XAMBIOA,TO,2,1,2020,January
24950,2020-06-08,1,XAMBIOA,TO,2,1,2020,June
46083,2020-10-05,0,XAMBIOA,TO,1,1,2020,October
